In [2]:
import numpy as np

np.random.seed(42)

# Time in minutes
time = np.arange(1440)

# Function to simulate sensor data with bounds
def generate_data(base, std_dev, size, min_val=None, max_val=None):
    data = base + np.random.normal(0, std_dev, size)
    if min_val is not None or max_val is not None:
        data = np.clip(data, min_val, max_val)
    return data

temperature = generate_data(22, 1.5, 1440)
humidity = generate_data(55, 5, 1440, 0, 100)
pressure = generate_data(1012, 3, 1440)

# Structured data using dictionary (alternative to column_stack)
sensor_data_alt = np.rec.fromarrays([time, temperature, humidity, pressure],
                                    names='time,temp,humidity,pressure')
print(sensor_data_alt[:3])


[(0, 22.74507123, 58.31440634, 1005.2853063 )
 (1, 21.79260355, 60.86736929, 1005.63789954)
 (2, 22.97153281, 55.90510779, 1010.17940447)]


In [4]:
import pandas as pd

df = pd.DataFrame({
    'Time': time,
    'Temperature': temperature,
    'Humidity': humidity,
    'Pressure': pressure
})

# Summary statistics
summary = df.describe().loc[['mean', 'min', 'max', 'std']]
print(summary)


             Time  Temperature   Humidity     Pressure
mean   719.500000    22.059766  55.132802  1011.887002
min      0.000000    17.138099  39.902439  1002.977103
max   1439.000000    27.779097  70.965538  1023.778713
std    415.836506     1.480999   4.887209     3.081130


In [5]:
df['Hour'] = df['Time'] // 60  # Convert minute to hour bin

hourly_means = df.groupby('Hour')[['Temperature', 'Humidity', 'Pressure']].mean()

print(hourly_means.head(3))  # First 3 hours


      Temperature   Humidity     Pressure
Hour                                     
0       21.768018  56.349887  1012.048283
1       21.994475  55.000500  1011.970714
2       22.137871  54.370007  1011.219726


In [6]:
# Boolean mask for temperature > threshold
mask = temperature > 25
count_exceed = np.count_nonzero(mask)
min_temp_time = time[np.argmin(temperature)]

print(f"Count above 25°C: {count_exceed}")
print(f"Time of min temp: {min_temp_time} minutes")


Count above 25°C: 41
Time of min temp: 262 minutes


In [7]:
df.to_csv("sensor_data_alt.csv", index=False)
print("Data exported as 'sensor_data_alt.csv'")


Data exported as 'sensor_data_alt.csv'
